In [1]:
#Importando as bibliotecas
import pandas as pd
import os
import datetime as dt
import numpy as np
import numpy as np
from dateutil.relativedelta import relativedelta
import requests
from simple_salesforce import Salesforce
from datetime import datetime, timedelta, date

In [2]:
def ImovelSF(sf):
    imoveis = sf.query_all("SELECT Id, Name FROM Imovel__c")["records"]
    df_imoveis = pd.DataFrame(imoveis, index=None).drop(columns='attributes')
    df_imoveis = df_imoveis.rename(columns={'Name':'Conjunto'})
    return df_imoveis

In [11]:
def conexao_sfc():
    session = requests.Session()
    session.verify = False

    sf = Salesforce(
        username='username',
        password='password',
        security_token='security_token',
        client_id="client_id",
        instance='instance',
        session_id='',
        session=session
    )
    return sf

In [12]:
def last_day_of_month(any_day):
    # this will never fail
    # get close to the end of the month for any day, and add 4 days 'over'
    next_month = any_day.replace(day=28) + relativedelta(days=4)
    # subtract the number of remaining 'overage' days to get last day of current month, or said programattically said, the previous day of the first of next month
    return (next_month - relativedelta(days=next_month.day))

In [13]:
def selectPremissa(sf, where):
    result = sf.query_all("SELECT Id, Fase__c, Mes_referencia__c, Name FROM Premissas_comerciais__c {}".format(where) )["records"]
    df = pd.DataFrame(result, index=None).drop(columns='attributes')
    return df

In [14]:
def selectDetalhePremissaSF(sf, df_premissa):
    idPremissa = df_premissa.iloc[0]['Id']
    result = sf.query_all("SELECT Id, Name, Premissas_comerciais__c, Carencia__c, Conjunto__c, Data_de_locacao__c, Descricao__c, Indice__c, Responsavel__c, Valor_por_m2_1_ano__c, Valor_por_m2_2_ano__c, Valor_por_m2_3_ano__c, Valor_por_m2_4_ano__c, Valor_por_m2_5_ano__c, Status_imovel__c, Previsao_saida__c FROM detalhes_das_premissas__c WHERE Premissas_comerciais__c = '{id}'".format(id=idPremissa))["records"]
    df = pd.DataFrame(result, index=None).drop(columns='attributes')
    return df

In [15]:
def selecionaPremissaAnterior(sf, df_premissa, df_detalhes):
    #df = pd.merge(df_detalhes, df_premissa, how="inner", left_on="Premissas_comerciais__c", right_on="Id")
    dataAnt = df_premissa.iloc[0]['Mes_referencia__c']
    dataAnt = datetime.strptime(dataAnt, "%Y-%m-%d").date() - relativedelta(months = 1)
    df_premissaAnt = selectPremissa(sf, 'WHERE Mes_referencia__c >= {dataini} and Mes_referencia__c <= {datafim}'.format(dataini=dataAnt.replace(day=1), datafim=last_day_of_month(dataAnt)))
    df_detalheAnt = selectDetalhePremissaSF(sf, df_premissaAnt)
    return df_premissaAnt, df_detalheAnt

In [16]:
def relacionaPremissa(df_detalhes, df_detalheAnt):
    df = pd.merge(df_detalhes, df_detalheAnt, how="left", left_on="Conjunto__c", right_on="Conjunto__c")
    df_junto = df.rename(columns={"Data_de_locacao__c_y":"Locação_antigo", "Data_de_locacao__c_x":"Locação_novo"})

    #Diferença de datas de locação 
    df_junto["Locação_novo"] = pd.to_datetime(df_junto['Locação_novo'], format="%Y-%m-%d", cache=False)
    df_junto["Locação_antigo"] = pd.to_datetime(df_junto['Locação_antigo'], format="%Y-%m-%d", cache=False)
    df_junto["diferenca_data"] = df_junto["Locação_novo"]-df_junto["Locação_antigo"] 

    # Sequência de datas para o arquivo separado
    min_antigo = min(df_junto["Locação_antigo"])
    min_novo = min(df_junto["Locação_novo"])
    max_antigo = max(df_junto["Locação_antigo"])
    max_novo = max(df_junto["Locação_novo"])
    max_tot = max([max_antigo,max_novo])
    min_tot = min([min_antigo,min_novo])
    
    print(df_junto)

In [17]:
#Função para retornar a classificação entre diferenças de datas
def diferenca(time):
    time = list(time)
    na = pd.isna(time)
    v = []
    for i in range(len(time)):
        if na[i]:
            v.append("Novo")
        else:
            dife = time[i]/dt.timedelta(days = 30)
            if dife < 0:
                v.append("Adiantou")
            elif dife > 0:
                v.append("Postergou")
            elif dife == 0:
                v.append("Igual")
    return(v)

In [18]:
#Gerando a conexão e obtendo os dados necessários
sf = conexao_sfc()
df_premissa = selectPremissa(sf, "WHERE Fase__c = 'Geração de relatório'")
df_detalhes = selectDetalhePremissaSF(sf, df_premissa)
conjuntos = ImovelSF(sf)
df_premissaAnt, df_detalheAnt = selecionaPremissaAnterior(sf, df_premissa, df_detalhes)

C:\Users\User\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'login.salesforce.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'kinea.my.salesforce.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'kinea.my.salesforce.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\User\anaconda3\lib\site-package

In [19]:
#Obtendo os dados das áreas locáveis atuais dos imóveis
areas = pd.DataFrame(sf.query_all("SELECT Id,area_locavel__c from Imovel__c")["records"],index = None)
areas.drop(columns = "attributes",inplace = True)
areas.rename(columns = {"area_locavel__c":"Area_locavel__c","Id":"ID Salesforce"},inplace = True)

C:\Users\User\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'kinea.my.salesforce.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [20]:
#Relacionando os conjuntos do data.frame atual
df_detalhes1 = pd.merge(df_detalhes,conjuntos,how = "left",left_on = "Conjunto__c",right_on = "Id")
df_detalhes1.drop(columns=["Id_y","Id_x"],inplace = True)
df_detalhes1.rename(columns = {"Conjunto__c":"ID Salesforce","Conjunto":"Conjunto__c"},inplace = True)
df_detalhes1 = pd.merge(df_detalhes1,areas,how = "left",on = "ID Salesforce")
#Relacionando os conjuntos do data.frame anterior
df_detalheAnt1 = pd.merge(df_detalheAnt,conjuntos,how = "left",left_on = "Conjunto__c",right_on = "Id")
df_detalheAnt1.drop(columns=["Id_y"],inplace = True)
df_detalheAnt1.rename(columns = {"Conjunto__c":"ID Salesforce","Conjunto":"Conjunto__c"},inplace = True)

In [21]:
#Selecionando as colunas necessárias
colunas = ["ID Salesforce","Conjunto__c","Descricao__c","Data_de_locacao__c","Valor_por_m2_1_ano__c"]
excel_antigo = df_detalheAnt1[colunas]
excel_antigo = excel_antigo.rename(columns = {"Valor_por_m2_1_ano__c":"Valor_por_m2_1_ano_antigo__c",
                               "Data_de_locacao__c":"Data_de_locacao_antigo__c"})
colunas.append("Area_locavel__c")
colunas.append("Carencia__c")
excel_novo = df_detalhes1[colunas]
colunas = colunas[0:3]

In [22]:
#Juntando os dois arquivos
excel_junto = pd.merge(excel_novo,excel_antigo,how = "left",copy=True,on = colunas)
z = []
for i in range(excel_junto.shape[0]):
    if all(pd.isna(excel_junto.iloc[i,])):
        z.append(i)
excel_junto = excel_junto.drop(z) #Retirando todas as linhas que tenham somente NAs
excel_junto["Data_de_locacao__c"] = [dt.datetime.strptime(i,"%Y-%m-%d") for i in excel_junto["Data_de_locacao__c"]]
excel_junto["Data_de_locacao_antigo__c"] = [dt.datetime.strptime(i,"%Y-%m-%d") for i in excel_junto["Data_de_locacao_antigo__c"]]
excel_junto["diferenca_data"] = excel_junto["Data_de_locacao__c"]-excel_junto["Data_de_locacao_antigo__c"] #Diferença de datas de locação

In [23]:
#Sequência de datas para o arquivo separado
min_antigo = min(excel_junto["Data_de_locacao_antigo__c"])
min_novo = min(excel_junto["Data_de_locacao__c"])
max_antigo = max(excel_junto["Data_de_locacao_antigo__c"])
max_novo = max(excel_junto["Data_de_locacao__c"])
max_tot = max([max_antigo,max_novo])
min_tot = min([min_antigo,min_novo])
datas = []
current = min_tot
while current <= max_tot:
    datas.append(current)
    current += relativedelta(months = 1)
for i in range(len(datas)):
    datas[i] = datas[i].strftime("%m/%Y")

In [24]:
#Criando outra coluna para ordenar o data.frame final
excel_junto["Locacao"] = excel_junto["Data_de_locacao__c"]  #Mudança
#Formatando as datas para %m/%Y
array = list(excel_junto["Data_de_locacao__c"].apply(str))
array1 = list(excel_junto["Data_de_locacao_antigo__c"].apply(str))
for i in range(len(array)):
    array[i] = dt.datetime.strptime(array[i],"%Y-%m-%d %H:%M:%S")
    array[i] = array[i].strftime("%m/%Y")
    if not array1[i] == "NaT":
        array1[i] = dt.datetime.strptime(array1[i],"%Y-%m-%d %H:%M:%S")
        array1[i] = array1[i].strftime("%m/%Y")
excel_junto["Data_de_locacao__c"] = array
excel_junto["Data_de_locacao_antigo__c"] = array1
excel_junto["diferenca_data"] = diferenca(excel_junto["diferenca_data"]) #Classificação de diferenças de datas
excel_junto["diferenca_valor"] = excel_junto["Valor_por_m2_1_ano__c"]-excel_junto["Valor_por_m2_1_ano_antigo__c"] #Diferença de valores

In [25]:
#Separando os dados, colocando os valores por m2 nas colunas das datas
excel_separado = excel_junto.pivot(index = ["ID Salesforce","Conjunto__c","Descricao__c",
                                            "Locacao","Carencia__c","Data_de_locacao__c",
                                            "Area_locavel__c","Valor_por_m2_1_ano__c","diferenca_data","diferenca_valor"],
                                   columns = "Data_de_locacao_antigo__c",values = "Valor_por_m2_1_ano_antigo__c").reset_index() #Mudança
if any(excel_separado.columns == "NaT"):
    excel_separado.drop(columns = ["NaT"])
excel_separado = excel_separado.pivot(index = ["ID Salesforce","Conjunto__c","Descricao__c",
                                               "Locacao","Carencia__c","Area_locavel__c",
                                               "diferenca_data","diferenca_valor"],
                                      columns = "Data_de_locacao__c",values="Valor_por_m2_1_ano__c").reset_index() #Mudança

In [32]:
#Criando colunas para todos as datas entre a menor e a maior data de locação
excel_separado1 = excel_separado.iloc[0:,0:7] #Mudança
nomes = excel_separado.columns[3:]
for i in datas:
    if i in nomes:
        excel_separado1 = pd.concat([excel_separado1,excel_separado[i]],axis = 1)
    else:
        excel_separado1[i] = np.NaN

In [33]:
#Inserindo os valores antigos e atuais nas colunas
for i in range(excel_separado1.shape[0]):
    cod = excel_separado1["ID Salesforce"][i]
    #Pegando os dados de valores antigos e atuais dos imóveis
    if pd.isna(cod):
        df = excel_junto[pd.isna(excel_junto["ID Salesforce"])]
    else:
        df = excel_junto[excel_junto["ID Salesforce"] == cod]
    diferenca = df["diferenca_data"]
    #Inserindo os valores nas respectivas colunas caso haja diferenças entre as datas
    if not (diferenca.values in ["Igual","Novo"]):
        loc_antigo = np.where([j == df["Data_de_locacao_antigo__c"].values[0] for j in excel_separado1.columns]) #Índice da coluna de valores antigos
        val_antigo = df["Valor_por_m2_1_ano_antigo__c"].values #Valores antigos
        loc_novo = np.where([j == df["Data_de_locacao__c"].values[0] for j in excel_separado1.columns]) #Índice da coluna de valores atuais
        loc_antigo = loc_antigo[0][0]
        loc_novo = loc_novo[0][0]
        val_novo = df["Valor_por_m2_1_ano__c"].values #Valores novos
        excel_separado1.iloc[i,loc_antigo] = val_antigo[0]
        excel_separado1.iloc[i,loc_novo] = val_novo[0]

In [34]:
#Renomeando as colunas e ordenando o data.frame pela data de locação atual e imóvel
excel_separado1 = excel_separado1.rename(columns = {"Conjunto__c":"Conjunto","Descricao__c":"Descricao",
                                                    "diferenca_data":"Status","diferenca_valor":"Diferenca valor",
                                                    "Area_locavel__c":"Area locavel",
                                                    "Carencia__c":"Carencia"})
excel_separado1 = excel_separado1.sort_values(["Locacao","Descricao"]) #Mudança
excel_separado1.drop(columns = "Locacao",inplace = True) #Mudança